In [2]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

In [3]:
max_dataset_size = 100000;
waveform_length = 72;
nb_of_elements = 150000;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
nb_of_datasets = 45;
snr_from = 20;
snr_to = 100;
max_shift = 0;
use_horizontal_flip = False;
use_vertical_flip = False;
filter_type = 'high';
sampling_rate = 24000;
passband = np.array([100], dtype=int);
order = 1;



In [4]:
root_folder = os.path.dirname(os.getcwd())
path_to_recordings = path.join(root_folder,'data/synthesized');
path_to_ground_truth_data = path.join(root_folder,'data/synthesized');
path_to_train_data = path.join(root_folder,'data/train_data_re_id.npy');
path_to_train_labels = path.join(root_folder,'data/train_labels_re_id.npy');
sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn
transform_list =[cnn.FilterSignalUsingButtersWorth(filter_type, sampling_rate, passband, order), cnn.OptimizedZScoreNormalizaton()];



In [5]:

# counts each class
classes_per_recording = np.zeros((nb_of_datasets), dtype='int')
for i in range(0 , nb_of_datasets):
    single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
    gt_data = np.load(single_recording_ground_truth);
    classes_per_recording[i] = np.unique(gt_data[1,:]).size;
    print(np.unique(gt_data[1,:]))
    print(np.min(gt_data))
print(classes_per_recording)

# class counter to concatenate classes together from different recordings
class_counter = np.zeros((nb_of_datasets, 1), dtype='int');
class_counter[0] = 0;
for i in range(1, nb_of_datasets):
    class_counter[i] = class_counter[i-1] + classes_per_recording[i - 1] - 1; 
print(class_counter)



[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
0.0
[0. 1. 2. 3. 4.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
0.0
[0. 1. 2.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
0.0
[0. 1. 2. 3.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0
[0. 1. 2. 

In [6]:
# couts sample count for each class
nb_of_classes = int(np.sum(classes_per_recording - 1) + 1);


In [7]:
print(classes_per_recording)
print(class_counter)

[17 20 13  5 16 12 18  3 20 21 21 21 11  4 10  9 10  8 19 15  5  7 18  7
 17 14  6 15  4  6 12 11  5  9 13 19 11 14  3  8 16 10 17 14  9]
[[  0]
 [ 16]
 [ 35]
 [ 47]
 [ 51]
 [ 66]
 [ 77]
 [ 94]
 [ 96]
 [115]
 [135]
 [155]
 [175]
 [185]
 [188]
 [197]
 [205]
 [214]
 [221]
 [239]
 [253]
 [257]
 [263]
 [280]
 [286]
 [302]
 [315]
 [320]
 [334]
 [337]
 [342]
 [353]
 [363]
 [367]
 [375]
 [387]
 [405]
 [415]
 [428]
 [430]
 [437]
 [452]
 [461]
 [477]
 [490]]


In [8]:
16+19+12

47

In [9]:
max_dataset_size = max_dataset_size + nb_of_classes - max_dataset_size % nb_of_classes;
samples_per_class = max_dataset_size // nb_of_classes;
print(samples_per_class)
print(max_dataset_size / nb_of_classes)
print(max_dataset_size)


201
201.0
100299


In [10]:
unique_list = [];
dataset = np.empty((1 , 1 ,waveform_length))
labels = np.empty((1, 0), dtype = "int")
# adds single unit activity data
for i in range(0, nb_of_datasets):
    added_sample_counter = np.zeros((classes_per_recording[i] - 1));
    data_iter_counter = 0;
    total_nb_of_samples = int((classes_per_recording[i] - 1) * samples_per_class);
    temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
    temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
    print(dataset.shape)
    # generates augmentations until there are same sized samples for each class
    while (data_iter_counter < total_nb_of_samples):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        dataset_spikes = cnn.GenerateDataset(single_recording_path, single_recording_ground_truth, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(dataset_spikes):
            neuron_index = target.item();
            if(neuron_index > 0 and added_sample_counter[neuron_index - 1] < samples_per_class):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = class_counter[i] + neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                added_sample_counter[neuron_index - 1] = added_sample_counter[neuron_index - 1] + 1;
                data_iter_counter = data_iter_counter + 1;
    unique_list.append(np.unique(temp_labels));
    print(added_sample_counter)
    dataset = np.concatenate((dataset, temp_spikes), axis=0)            
    labels = np.concatenate((labels, temp_labels), axis=1)            

    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(samples_per_class);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        dataset_spikes = cnn.GenerateDataset(single_recording_path, single_recording_ground_truth, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(dataset_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
unique_list.append(np.unique(temp_labels));

dataset = np.concatenate((dataset, temp_spikes), axis=0)    
    
labels = np.concatenate((labels, temp_labels), axis=1)            


(1, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  72.25430928517251
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b260f4048>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(16, dtype=torch.int32)


/home/vtpc/Documents/Alvils/spike-sorting/custom_resnet/CustomResnet.py:293: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  temp = temp.new_tensor(data);


dataset len:  12008
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  52.03211877004949
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05f96c88>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(16, dtype=torch.int32)
dataset len:  12008
[201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201.
 201. 201.]
(3217, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  28.060626481280398
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05f96cf8>, <custom_resnet.CustomResnet.FilterSignalUsingBut

dataset len:  13995
[201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201. 201.
 201. 201. 201.]
(18895, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  54.33590135790255
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05f96c88>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(2, dtype=torch.int32)
dataset len:  3489
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  85.44129320231578
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b260f4048>, <custom_resnet.CustomResnet.FilterSignalUsing

[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fb4ba8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(10, dtype=torch.int32)
dataset len:  7790
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  88.1685189057336
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fb4da0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(10, dtype=torch.int32)
dataset len:  7790
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  39.03672836433749
flip_data_horz:  0
[ 0

...

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcc128>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(4, dtype=torch.int32)
dataset len:  4166
[201. 201. 201. 201.]
(73768, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  35.837485253119326
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcc5c0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(8, dtype=torch.int32)
dataset len:  6848
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_inde

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcc5c0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(7, dtype=torch.int32)
dataset len:  7354
[201. 201. 201. 201. 201. 201. 201.]
(87838, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  57.397286026091564
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fccdd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(15, dtype=torch.int32)
dataset len:  13344
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_st

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcc438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(8, dtype=torch.int32)
dataset len:  6663
[201. 201. 201. 201. 201. 201. 201. 201.]
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  47.12731622087304
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcc438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(16, dtype=torch.int32)
dataset len:  12008
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shif

dataset len:  9682
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  81.96437190401028
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcca90>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(8, dtype=torch.int32)
dataset len:  7552
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  44.10784205000869
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fcca90>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object a

dataset len:  8222
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  46.03643834480034
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fccdd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f9b260f40b8>]
tensor(10, dtype=torch.int32)
dataset len:  9075
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  0
shift_to:  1
shift_step:  1
shift_indexes:  tensor([0], dtype=torch.int32)
snr_ratio:  71.07846678736992
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f9b05fccdd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f9b260eef98>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object 

In [14]:
for i in range(0, len(unique_list)):
    print(unique_list[i])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35]
[36 37 38 39 40 41 42 43 44 45 46 47]
[48 49 50 51]
[52 53 54 55 56 57 58 59 60 61 62 63 64 65 66]
[67 68 69 70 71 72 73 74 75 76 77]
[78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94]
[95 96]
[ 97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114
 115]
[116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135]
[136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155]
[156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
 174 175]
[176 177 178 179 180 181 182 183 184 185]
[186 187 188]
[189 190 191 192 193 194 195 196 197]
[198 199 200 201 202 203 204 205]
[206 207 208 209 210 211 212 213 214]
[215 216 217 218 219 220 221]
[222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239]
[240 241 242 243 244 245 246 247 248 249 250 251 252 253]
[254 255 256 257]
[258 259 260 261 262 2

In [15]:
class_sample_count = np.unique(labels, return_counts=True)[1]
class_sample_count

array([201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 20

In [16]:
import matplotlib.pyplot as plt
rnd = np.random.randint(2480000, 2500000)
print(labels[0, rnd])
plt.plot(dataset[rnd, :, :].ravel())

IndexError: index 2497291 is out of bounds for axis 1 with size 100299

In [19]:
np.save(path_to_train_data, dataset)
np.save(path_to_train_labels, labels)


In [18]:
mean = (np.mean(np_data_spikes))
std = (np.std(np_data_spikes))
np.save(path_to_mean_std, [mean, std])

print(mean)
print(std)

NameError: name 'np_data_spikes' is not defined